# This is an Updated version of my previous public kernel <https://www.kaggle.com/kushal1506/moa-pytorch-0-01859-rankgauss-pca-nn>

## Updates -

* Implementing Feature Engineering 
* Implementing Label Smoothing

# **If U find it helpful and consider forking , please do Upvote** :)

In [1]:
import sys
sys.path.append('../input/iterativestratification')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

%mkdir model
%mkdir interim
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA,FactorAnalysis

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.preprocessing import QuantileTransformer

In [4]:
NB = '25'

IS_TRAIN = True
MODEL_DIR = "model" # "../model"
INT_DIR = "interim" # "../interim"


In [5]:
os.listdir('../input/lish-moa')

['sample_submission.csv',
 'train_drug.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'test_features.csv']

In [6]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drugs = pd.read_csv('../input/lish-moa/train_drug.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
train_features = train_drugs.merge(train_features,on='sig_id',how='left')
#train_target_scored = train_target_scored.merge(train_drug,on='sig_id')

In [7]:
train_features

,sig_id,drug_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,b68db1d53,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,df89a8e5a,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,18bb41b2c,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,8c7f86626,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,7cbed3131,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,df1d0a5a1,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
23810,id_fffb70c0c,ecf3b6b74,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
23811,id_fffc1c3f4,cacb2b860,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808
23812,id_fffcb9e7c,8b87a7a83,trt_cp,24,D1,0.6660,0.2324,0.4392,0.2044,0.8531,...,-0.1105,0.4258,-0.2012,0.1506,1.5230,0.7101,0.1732,0.7015,-0.6290,0.0740


In [8]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [9]:
#RankGauss

for col in (GENES + CELLS):

    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    if IS_TRAIN:
        transformer = QuantileTransformer(n_quantiles=100, random_state=123, output_distribution="normal")
        transformer.fit(raw_vec)
        pd.to_pickle(transformer, f'{MODEL_DIR}/{NB}_{col}_quantile_transformer.pkl')
    else:
        transformer = pd.read_pickle(f'{MODEL_DIR}/{NB}_{col}_quantile_transformer.pkl')        


    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [10]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [11]:
# GENES
n_comp = 90  #<--Update

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
#data2 = (FactorAnalysis(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
if IS_TRAIN:
    fa = FactorAnalysis(n_components=n_comp, random_state=1903).fit(data[GENES])
    pd.to_pickle(fa, f'{MODEL_DIR}/{NB}_factor_analysis_g.pkl')
    #umap = UMAP(n_components=n_dim, random_state=1903).fit(data[GENES])
    #pd.to_pickle(umap, f'{MODEL_DIR}/{NB}_umap_g.pkl')
else:
    fa = pd.read_pickle(f'{MODEL_DIR}/{NB}_factor_analysis_g.pkl')
    #umap = pd.read_pickle(f'{MODEL_DIR}/{NB}_umap_g.pkl')
data2 = fa.transform(data[GENES])
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [12]:
#CELLS
n_comp = 50  #<--Update

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
if IS_TRAIN:
    fa = FactorAnalysis(n_components=n_comp, random_state=1903).fit(data[CELLS])
    pd.to_pickle(fa, f'{MODEL_DIR}/{NB}_factor_analysis_c.pkl')
    #umap = UMAP(n_components=n_dim, random_state=1903).fit(data[GENES])
    #pd.to_pickle(umap, f'{MODEL_DIR}/{NB}_umap_g.pkl')
else:
    fa = pd.read_pickle(f'{MODEL_DIR}/{NB}_factor_analysis_c.pkl')
    #umap = pd.read_pickle(f'{MODEL_DIR}/{NB}_umap_g.pkl')
data2 = fa.transform(data[CELLS])
#data2 = (FactorAnalysis(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [13]:
train_features.shape

(23814, 1017)

In [14]:
train_features

,sig_id,drug_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,...,pca_C-40,pca_C-41,pca_C-42,pca_C-43,pca_C-44,pca_C-45,pca_C-46,pca_C-47,pca_C-48,pca_C-49
0,id_000644bb2,b68db1d53,trt_cp,24,D1,1.134849,0.907687,-0.416385,-0.966814,-0.254723,...,0.267100,0.279708,-0.473689,0.183542,0.472832,-0.193523,-0.120138,-0.034791,0.310455,-0.153980
1,id_000779bfc,df89a8e5a,trt_cp,72,D1,0.119282,0.681738,0.272399,0.080113,1.205169,...,0.001024,-0.060906,-0.237748,0.163810,0.201246,0.057059,0.097315,-0.239799,0.153507,0.167928
2,id_000a6266a,18bb41b2c,trt_cp,48,D1,0.779973,0.946463,1.425350,-0.132928,-0.006122,...,-0.122583,-0.422981,0.176960,0.460475,-0.081269,0.202796,-0.063203,-0.091870,0.153470,-0.113158
3,id_0015fd391,8c7f86626,trt_cp,48,D1,-0.734910,-0.274641,-0.438509,0.759097,2.346330,...,-0.298000,-0.084247,0.092568,0.119792,-0.093275,-0.004222,0.098613,-0.007616,0.119951,0.043282
4,id_001626bd3,7cbed3131,trt_cp,72,D2,-0.452718,-0.477513,0.972316,0.970731,1.463427,...,0.133030,-0.325759,-0.398813,-0.122868,-0.363659,0.081824,-0.576945,0.044400,0.394367,0.115312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,df1d0a5a1,trt_cp,24,D2,0.209361,-0.022389,-0.235888,-0.796989,-0.674009,...,0.337849,0.278922,-0.070416,0.058629,0.078973,0.318525,-0.110655,0.165577,-0.099570,-0.185534
23810,id_fffb70c0c,ecf3b6b74,trt_cp,24,D2,-1.911021,0.587228,-0.588417,1.296405,-1.002640,...,-1.228912,0.328819,-0.224856,0.251855,-0.039027,0.375471,-0.076218,0.035372,-0.300387,-0.331585
23811,id_fffc1c3f4,cacb2b860,ctl_vehicle,48,D2,0.525149,0.631225,0.288173,-1.139968,0.769550,...,-0.017706,0.130790,-0.670019,0.136750,-0.181211,0.188200,-0.236776,-0.307762,-0.014583,0.131660
23812,id_fffcb9e7c,8b87a7a83,trt_cp,24,D1,0.816407,0.417618,0.431631,0.300617,1.070346,...,-0.286574,-0.272079,0.443892,-0.099474,-0.447661,-0.084755,0.319817,-0.089484,0.030659,0.332977


In [15]:
from sklearn.feature_selection import VarianceThreshold


#var_thresh = VarianceThreshold(0.8)  #<-- Update
var_thresh = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")

data = train_features.append(test_features)
if IS_TRAIN:
    transformer = QuantileTransformer(n_quantiles=100, random_state=123, output_distribution="normal")
    transformer.fit(data.iloc[:,5:])
    pd.to_pickle(transformer, f'{MODEL_DIR}/{NB}_{col}_quantile_transformer2.pkl')
else:
    transformer = pd.read_pickle(f'{MODEL_DIR}/{NB}_{col}_quantile_transformer2.pkl')  
data_transformed = transformer.transform(data.iloc[:, 5:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape


(23814, 1016)

In [16]:
from sklearn.cluster import KMeans
def fe_cluster(train, test, n_clusters_g = 45, n_clusters_c = 15, SEED = 123):
    
    features_g = list(train.columns[4:776])
    features_c = list(train.columns[776:876])
    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis = 0)
        kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

#train_features ,test_features=fe_cluster(train_features,test_features)

In [17]:
def fe_stats(train, test):
    
    features_g = list(train.columns[4:776])
    features_c = list(train.columns[776:876])
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

In [18]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target = train[train_targets_scored.columns]

In [19]:
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

In [20]:
train

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.147013,0.901837,-0.418317,-0.961328,-0.254748,-1.021168,-1.368753,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.128837,0.676615,0.274303,0.090277,1.208714,0.688985,0.316750,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.790059,0.940310,1.427325,-0.121422,-0.001534,1.494980,0.238795,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.729887,-0.277122,-0.441238,0.766535,2.327042,-0.862822,-2.304780,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.444200,-0.481230,0.974570,0.977613,1.468226,-0.874366,-0.372617,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.247854,-1.231353,0.221655,-0.354349,-0.332003,0.570609,-0.150057,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.217631,-0.026850,-0.237436,-0.787443,-0.677697,0.919339,0.743390,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,24,D2,-1.911045,0.581948,-0.588666,1.303231,-1.008858,0.851933,-0.302789,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.826571,0.411264,0.433236,0.307423,1.075457,-0.024437,0.051466,...,0,0,0,0,0,0,0,0,0,0


In [21]:
target_cols = target.drop('sig_id', axis=1).columns.values.tolist()

In [22]:
#folds = train.copy()

#mskf = MultilabelStratifiedKFold(n_splits=5)

#for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
#    folds.loc[v_idx, 'kfold'] = int(f)

#folds['kfold'] = folds['kfold'].astype(int)
#folds

In [23]:
train

,sig_id,cp_time,cp_dose,0,1,2,3,4,5,6,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,24,D1,1.147013,0.901837,-0.418317,-0.961328,-0.254748,-1.021168,-1.368753,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,72,D1,0.128837,0.676615,0.274303,0.090277,1.208714,0.688985,0.316750,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,48,D1,0.790059,0.940310,1.427325,-0.121422,-0.001534,1.494980,0.238795,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,48,D1,-0.729887,-0.277122,-0.441238,0.766535,2.327042,-0.862822,-2.304780,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,72,D2,-0.444200,-0.481230,0.974570,0.977613,1.468226,-0.874366,-0.372617,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,72,D1,0.247854,-1.231353,0.221655,-0.354349,-0.332003,0.570609,-0.150057,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,24,D2,0.217631,-0.026850,-0.237436,-0.787443,-0.677697,0.919339,0.743390,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,24,D2,-1.911045,0.581948,-0.588666,1.303231,-1.008858,0.851933,-0.302789,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,24,D1,0.826571,0.411264,0.433236,0.307423,1.075457,-0.024437,0.051466,...,0,0,0,0,0,0,0,0,0,0


In [24]:
train = train_drugs.merge(train,on='sig_id')

In [25]:
train[['sig_id','drug_id']]

,sig_id,drug_id
0,id_000644bb2,b68db1d53
1,id_000779bfc,df89a8e5a
2,id_000a6266a,18bb41b2c
3,id_0015fd391,8c7f86626
4,id_001626bd3,7cbed3131
...,...,...
21943,id_fff8c2444,6c3a459be
21944,id_fffb1ceed,df1d0a5a1
21945,id_fffb70c0c,ecf3b6b74
21946,id_fffcb9e7c,8b87a7a83


In [26]:
scored = (train[['sig_id','drug_id']]).merge(train[train_targets_scored.columns],on='sig_id')
scored

,sig_id,drug_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,b68db1d53,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,df89a8e5a,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,18bb41b2c,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,8c7f86626,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,7cbed3131,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,df1d0a5a1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21945,id_fffb70c0c,ecf3b6b74,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21946,id_fffcb9e7c,8b87a7a83,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
targets = scored.columns[2:]

In [28]:
FOLDS = 5
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index
vc2 = vc.loc[vc>18].index
folds = train.copy()
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=34)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)
                
folds['kfold'] = folds.drug_id.map(dct1)
folds.loc[folds.kfold.isna(),'kfold'] =\
    folds.loc[folds.kfold.isna(),'sig_id'].map(dct2)
folds.kfold = folds.kfold.astype('int8')

In [29]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target.shape)
print(sample_submission.shape)

(21948, 1237)
(21948, 1238)
(3624, 1030)
(21948, 207)
(3982, 207)


In [30]:
folds

,sig_id,drug_id,cp_time,cp_dose,0,1,2,3,4,5,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_000644bb2,b68db1d53,24,D1,1.147013,0.901837,-0.418317,-0.961328,-0.254748,-1.021168,...,0,0,0,0,0,0,0,0,0,2
1,id_000779bfc,df89a8e5a,72,D1,0.128837,0.676615,0.274303,0.090277,1.208714,0.688985,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,18bb41b2c,48,D1,0.790059,0.940310,1.427325,-0.121422,-0.001534,1.494980,...,0,0,0,0,0,0,0,0,0,2
3,id_0015fd391,8c7f86626,48,D1,-0.729887,-0.277122,-0.441238,0.766535,2.327042,-0.862822,...,0,0,0,0,0,0,0,0,0,3
4,id_001626bd3,7cbed3131,72,D2,-0.444200,-0.481230,0.974570,0.977613,1.468226,-0.874366,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,6c3a459be,72,D1,0.247854,-1.231353,0.221655,-0.354349,-0.332003,0.570609,...,0,0,0,0,0,0,0,0,0,1
21944,id_fffb1ceed,df1d0a5a1,24,D2,0.217631,-0.026850,-0.237436,-0.787443,-0.677697,0.919339,...,0,0,0,0,0,0,0,0,0,1
21945,id_fffb70c0c,ecf3b6b74,24,D2,-1.911045,0.581948,-0.588666,1.303231,-1.008858,0.851933,...,0,0,0,0,0,0,0,0,0,3
21946,id_fffcb9e7c,8b87a7a83,24,D1,0.826571,0.411264,0.433236,0.307423,1.075457,-0.024437,...,0,0,0,0,0,0,0,0,0,0


# Dataset Classes

In [31]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct
    

In [32]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
#         print(inputs.shape)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds
   
    

In [33]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [34]:
class Model(nn.Module):      # <-- Update
    def __init__(self, num_features, num_targets, hidden_size):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.25)
        self.dense2 = nn.Linear(hidden_size, hidden_size)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [35]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [36]:
feature_cols = [c for c in process_data(folds).columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold','sig_id','drug_id']]
len(feature_cols)

1032

In [37]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
BATCH_SIZE = 128
LEARNING_RATE = 5e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5            #<-- Update
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

num_features=len(feature_cols)
num_targets=len(target_cols)
hidden_size=2048


In [38]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)
    
    x_train, y_train  = train_df[feature_cols].values, train_df[target_cols].values
    x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols].values
    
    train_dataset = MoADataset(x_train, y_train)
    valid_dataset = MoADataset(x_valid, y_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"SEED: {seed} ,FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}")
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"model/{NB}-scored2-SEED{seed}-FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets=num_targets,
        hidden_size=hidden_size,

    )
    
    model.load_state_dict(torch.load(f"model/{NB}-scored2-SEED{seed}-FOLD{fold}_.pth"))
    model.to(DEVICE)
    
 #   if not IS_TRAIN:
   # valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
   # oof[val_idx] = valid_preds     
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [39]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [40]:
# Averaging on multiple SEEDS

SEED = [940, 1513, 1269,1392,1119,1303]  #<-- Update
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions


SEED: 940, FOLD: 0, EPOCH: 0, train_loss: 0.47159688129857946
SEED: 940 ,FOLD: 0, EPOCH: 0, valid_loss: 0.025318066296832902
SEED: 940, FOLD: 0, EPOCH: 1, train_loss: 0.023800494584397678
SEED: 940 ,FOLD: 0, EPOCH: 1, valid_loss: 0.019906335696578025
SEED: 940, FOLD: 0, EPOCH: 2, train_loss: 0.021582841546866145
SEED: 940 ,FOLD: 0, EPOCH: 2, valid_loss: 0.018442604690790176
SEED: 940, FOLD: 0, EPOCH: 3, train_loss: 0.02066543649365432
SEED: 940 ,FOLD: 0, EPOCH: 3, valid_loss: 0.01816304475069046
SEED: 940, FOLD: 0, EPOCH: 4, train_loss: 0.02002081629840562
SEED: 940 ,FOLD: 0, EPOCH: 4, valid_loss: 0.017897970655134748
SEED: 940, FOLD: 0, EPOCH: 5, train_loss: 0.019894176373516557
SEED: 940 ,FOLD: 0, EPOCH: 5, valid_loss: 0.018028783957873072
SEED: 940, FOLD: 0, EPOCH: 6, train_loss: 0.01989501514845956
SEED: 940 ,FOLD: 0, EPOCH: 6, valid_loss: 0.018118914110319954
SEED: 940, FOLD: 0, EPOCH: 7, train_loss: 0.01990952098021542
SEED: 940 ,FOLD: 0, EPOCH: 7, valid_loss: 0.01788411276148898

In [41]:
train_targets_scored

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23809,id_fffb1ceed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23810,id_fffb70c0c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23811,id_fffc1c3f4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23812,id_fffcb9e7c,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
len(target_cols)


206

In [43]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)


y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0
for i in range(len(target_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target.shape[1]
    
print("CV log_loss: ", score)
    

CV log_loss:  0.015656427174735293


In [44]:
sub = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
sub.to_csv('submission.csv', index=False)

In [45]:
sub.shape

(3982, 207)